In [3]:
# installing libraries

!pip install unidecode -q
!pip install gensim==4.3.1 -q
!pip install fasttext
!pip install tensorflow



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 5.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.10.4-py3-none-any.whl (222 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp310-cp310-linux_x86_64.whl size=4393404 sha256=1850384be6214ca6038ff51d8ffccfde51773b4219e57712707ce8bca5687832
  Stored in directory: /root/.cache/pip/wheels/a5/13/75/f811c84a8ab36eedbaef977a6a58a98990e8e0f1967f98f394
Successfully built fasttext
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import gensim
import re
import logging
import numpy as np
import pandas as pd
import multiprocessing

from re import sub
from time import time 
from unidecode import unidecode
from gensim.models import Word2Vec
from collections import defaultdict
from gensim.models import KeyedVectors
from gensim.test.utils import get_tmpfile
from gensim.models.phrases import Phrases, Phraser

logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

In [5]:
#data loading
colnames=['id', 'comment'] 
hiphop = pd.read_csv("/content/drive/MyDrive/hiphopheads.csv", on_bad_lines='skip',engine='python', header = None, names = colnames)
jazz = pd.read_csv("/content/drive/MyDrive/jazz.csv", on_bad_lines='skip',engine='python', header = None, names = colnames)
punk = pd.read_csv("/content/drive/MyDrive/punk.csv", on_bad_lines='skip',engine='python', header = None, names = colnames)
metal = pd.read_csv("/content/drive/MyDrive/metal.csv", on_bad_lines='skip',engine='python', header = None, names = colnames)
classical = pd.read_csv("/content/drive/MyDrive/classicalmusic.csv", on_bad_lines='skip',engine='python', header = None, names = colnames)


In [ ]:
print(len(hiphop))
print(len(jazz))
print(len(punk))
print(len(metal))
print(len(classical))

145050
114320
140253
118590
112552


AttributeError: ignored

In [6]:
import re, string, unicodedata
import nltk
import inflect
nltk.download('stopwords')
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer



def preprocess(text):
    text = unidecode(text)
    text = str(text)
    text = text.lower()

    text = sub(r"[^A-Za-z0-9^,!?.\/'+]", " ", text)
    text = sub(r"\+", "", text)
    text = sub(r",", "", text)
    text = sub(r"\.", "", text)
    text = sub(r"!", "", text)
    text = sub(r"\?", "", text)
    text = sub(r"'", "", text)
    text = sub(r":", "", text)
    text = sub(r"\s{3,}", " ", text)

    # remove urls
    text = sub(r"http\S+", "", text)
    # split into words
    tokens = text.split()

    # remove stopwords
    words = []
    for word in tokens:
        if word not in stopwords.words('english'):
            words.append(word)


    # stem words
    stemmer = LancasterStemmer()
    stems = []
    for word in words:
        stem = stemmer.stem(word)
        stems.append(stem)

    # lemmatize
    lemmatizer = WordNetLemmatizer()
    lemmas = []
    for word in stems:
        lemma = lemmatizer.lemmatize(word, pos='v')
        lemmas.append(lemma)

    return lemmas 


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
def create_tfidf_dictionary(x, transformed_file, features):
    vector_coo = transformed_file[x.name].tocoo()
    vector_coo.col = features.iloc[vector_coo.col].values
    dict_from_coo = dict(zip(vector_coo.col, vector_coo.data))
    return dict_from_coo

def replace_tfidf_words(x, transformed_file, features):
    dictionary = create_tfidf_dictionary(x, transformed_file, features)   
    return list(map(lambda y:dictionary[f'{y}'], x.comment.split()))

def replace_sentiment_words(word, sentiment_dict):
    try:
        out = sentiment_dict[word]
    except KeyError:
        out = 0
    return out
  

In [17]:
# data preprocessing
import nltk
import re, string, unicodedata
import nltk
import fasttext
import pandas as pd
import tensorflow as tf
import keras
from keras.models import Sequential
from keras import layers
from tensorflow.keras.optimizers import RMSprop, Adam
from keras.preprocessing.text import Tokenizer
from keras import regularizers
from keras import backend as K
from keras.callbacks import ModelCheckpoint
from keras.utils import pad_sequences, to_categorical
from sklearn.model_selection import train_test_split

import inflect
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer

import nltk
import inflect
nltk.download('wordnet')
nltk.download('stopwords')
from nltk import word_tokenize, sent_tokenize

stop_words = stopwords.words('english')
#array = [punk, hiphop, jazz, metal, classical]
array = [punk.head(5)]
for genre in array:
  genre = genre.dropna().drop_duplicates().reset_index(drop=True)
  genre.comment = genre.comment.apply(lambda x: preprocess(x))
  genre_model = genre.copy()
  genre_model = genre_model[genre_model.comment.str.len()>1]
  #bigrams
  sent = [row for row in genre_model.comment]
  phrases = Phrases(sent, min_count=1, progress_per=50000)
  bigram = Phraser(phrases)
  sentences = bigram[sent]

  #word embedding
  ##word2vec
  w2v_model = Word2Vec(min_count=1,
                     window=4,
                     vector_size=300,
                     sample=1e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=multiprocessing.cpu_count()-1)
  w2v_model.build_vocab(sentences, progress_per=50000)
  w2v_model.train(sentences,
                total_examples=w2v_model.corpus_count,
                epochs=30,
                report_delay=1)
  w2v_model.init_sims(replace=True)
  w2v_model.save("word2vec.model")




  ## fast text
  with open('labeled_data.txt', 'w') as file:
    for comment in genre.comment:
        labeled_comment = f'__label__unknown {comment}\n'
        file.write(labeled_comment)

  model = fasttext.train_supervised('labeled_data.txt', lr=0.1, epoch=25, wordNgrams=2)
  model.save_model('fasttext_model.bin')


  #exporting data
  file_export = genre_model.copy()
  file_export['old_comment'] = file_export.comment
  file_export.old_comment = file_export.old_comment.str.join(' ')
  file_export.coment = file_export.comment.apply(lambda x: ' '.join(bigram[x]))
  file_export[['comment']].to_csv('cleaned_dataset.csv', index=False)



  ## kmeans word2vec
  from sklearn.cluster import KMeans
  word_vectors = Word2Vec.load("word2vec.model").wv
  model = KMeans(n_clusters=2,
               max_iter=1000,
               random_state=42,
               n_init=50)
  model.fit(X=word_vectors.vectors.astype('double'))
  positive_cluster_index = 1
  positive_cluster_center = model.cluster_centers_[positive_cluster_index]
  negative_cluster_center = model.cluster_centers_[1-positive_cluster_index]
  print(word_vectors)
  print(word_vectors.index_to_key)
  words = pd.DataFrame(word_vectors.index_to_key)
  words.columns = ['words']
  words['vectors'] = words.words.apply(lambda x: word_vectors[f'{x}'])
  words['cluster'] = words.vectors.apply(lambda x: model.predict([np.array(x)]))
  words.cluster = words.cluster.apply(lambda x: x[0])
  words['cluster_value'] = [1 if i==positive_cluster_index else -1 for i in words.cluster]
  words['closeness_score'] = words.apply(lambda x: 1/(model.transform([x.vectors]).min()), axis=1)
  words['sentiment_coeff'] = words.closeness_score * words.cluster_value
  words[['words', 'sentiment_coeff']].to_csv('sentiment_dictionary.csv', index=False)





  #labelling data
  from IPython.display import display
  from sklearn.feature_extraction.text import TfidfVectorizer
  from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
  final_file = pd.read_csv('cleaned_dataset.csv')
  sentiment_map = pd.read_csv('sentiment_dictionary.csv')
  sentiment_dict = dict(zip(sentiment_map.words.values, sentiment_map.sentiment_coeff.values))

  file_weighting = final_file.copy()
  tfidf = TfidfVectorizer(tokenizer=lambda y: y.split(), norm=None)
  tfidf.fit(file_weighting.comment)
  features = pd.Series(tfidf.get_feature_names_out())
  transformed = tfidf.transform(file_weighting.comment)
  replaced_tfidf_scores = file_weighting.apply(lambda x: replace_tfidf_words(x, transformed, features), axis=1)
  replaced_closeness_scores = file_weighting.comment.apply(lambda x: list(map(lambda y: replace_sentiment_words(y, sentiment_dict), x.split())))
  replacement_df = pd.DataFrame(data=[replaced_closeness_scores, replaced_tfidf_scores, file_weighting.comment]).T
  replacement_df.columns = ['sentiment_coeff', 'tfidf_scores', 'sentence']
  replacement_df['sentiment_rate'] = replacement_df.apply(lambda x: np.array(x.loc['sentiment_coeff']) @ np.array(x.loc['tfidf_scores']), axis=1)
  replacement_df['prediction'] = (replacement_df.sentiment_rate>0).astype('int8')
  




















[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
<ipython-input-17-75d81ee055ab>:59: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  w2v_model.init_sims(replace=True)
<ipython-input-17-75d81ee055ab>:79: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  file_export.coment = file_export.comment.apply(lambda x: ' '.join(bigram[x]))


KeyedVectors<vector_size=300, 34 keys>
['black_flag', 'song', 'ston', 'start', 'blink', 'day', 'deb', 'list', 'gelatin', 'first', 'count', 'ep', 'breakdown', 'nerv', 'ramon', 'word', '182', 'punk', 'sixteen', 'dammit', 'damn', 'good', 'breakup', 'eith', 'laur', 'jan', 'grac', 'kathleen', 'hannah', 'bush', 'x', 'girlfriend', 'last', 'two']


xxxxxxxxxxxxx
[[-0.00829935  0.00783223 -0.0062062  ...  0.00958373 -0.00548349
  -0.00187261]
 [ 0.00238093  0.00509113  0.00831976 ...  0.00242033 -0.00682412
  -0.00715008]
 [ 0.00730021 -0.00806438 -0.00091569 ...  0.00525279  0.00587604
   0.0088221 ]
 ...
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]]


ValueError: ignored